In [1]:
from tqdm import tqdm
import os
from datasets import load_dataset, concatenate_datasets
import json
import re
from copy import deepcopy

import asyncio
import math
import numpy as np
import random
import pandas as pd
from ollama import AsyncClient

from random import sample
from collections import Counter
from openai import OpenAI
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from utils import *

/root/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load the Wikihow dataset

In [7]:
wifi_dataset=load_dataset_dict('/workdir/important_datasets/AGRIVQA/WikiHow', concat=True)

In [19]:
wifi_dataset[0]['question'].split('"')[3]

'Prune Lilacs'

In [26]:
title = set()
for doc in wifi_dataset:
    section_title = doc['question'].split('"')[1]
    page_title = doc['question'].split('"')[3]
    title.add(f"{page_title}: {section_title}")
    if doc['question'].split('"')[1]=='Steps':
        print(section_title)

Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps
Steps


In [27]:
title

{'Prune Lilacs: Keeping Your Crabapple Tree Healthy',
 'Prune Lilacs: Caring for Cumin Plants',
 'Prune Lilacs: Decorating the Bottle',
 'Prune Lilacs: Propagating Clusia in Soil',
 'Prune Lilacs: Planting Rhizomes in Containers',
 'Prune Lilacs: Watering and Care',
 'Prune Lilacs: Keeping Ticks Out of Your Yard',
 'Prune Lilacs: Planting Your Geraniums',
 'Prune Lilacs: Building the House',
 'Prune Lilacs: Making Lavender Repellants',
 'Prune Lilacs: Removing and Drying the Leaves',
 'Prune Lilacs: Maintaining Your Birch Tree',
 'Prune Lilacs: Caring for an Anthurium',
 'Prune Lilacs: Targeting Certain Parts of the Plant',
 'Prune Lilacs: Recycling Empty Roundup Containers',
 'Prune Lilacs: Getting Rid of Lawn Mowers in Other Ways',
 'Prune Lilacs: Planting and Growing Oats',
 'Prune Lilacs: Caring for a Young Tree',
 'Prune Lilacs: Masculine Flower Tattoo Ideas',
 'Prune Lilacs: Pruning a Honeysuckle Shrub',
 'Prune Lilacs: Removing Morning Glory',
 'Prune Lilacs: Taking Cuttings fro

In [19]:
chat = Chat()

In [21]:
chat.model

'llama3.3'

In [22]:
await chat('vaffanculo')

"Sembra che tu abbia usato un'espressione piuttosto forte. Se hai bisogno di parlare di qualcosa o hai una domanda, sono qui per aiutarti in modo rispettoso e professionale. Come posso assisterti oggi?"